In [1]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model = AutoModelForCausalLM.from_pretrained ("numind/NuExtract-tiny",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained ("numind/NuExtract-tiny",trust_remote_code=True)
model.to("cuda")
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
      )
    )
    (norm): 

In [9]:
def predict_NuExtract(model, tokenizer, text, schema, example=["", ""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm = "<|input|>\n### Template:\n" + schema + "\n"

    for i in example:
        if i != "":
            input_llm += "### Example:\n" + json.dumps(json.loads(i), indent=4) + "\n"

    input_llm += "### Text:\n" + text + "\n</output/>\n"

    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)

    # Check if the tokens exist in the output before splitting
    if "</output |>" in output and "<<end-output|>" in output:
        return output.split("</output |>")[1].split("<<end-output|>")[0]
    else:
        # If the tokens are missing, return the full output or handle it accordingly
        return output

In [10]:
# Load OCR JSON output
with open("/content/new_bill_results.json", "r") as f:
    ocr_data = json.load(f)

# Extract the text from the OCR JSON (assuming it is located under 'text_lines')
text = " ".join([line['text'] for page in ocr_data.values() for line in page[0]['text_lines']])
print(text)

PL:F32771816 IT:1 Tax Invoice S:S444795365[1] Registered Office :: Tech Connect Retail Pvt. Ltd., 502 - 503, 5th Floor, Prospect Chanmers, Dr. d n Road, Fort Mumbai, Mumbai, MAHARASHTRA, INDIA-400001 Ordered Through Flipkart Sold by Tech Connect Retail Pvt. Ltd., GSTIN 27AAICA 4872D 120 Avaashya CCI Logistics Ltd Renaissance 0D112B39788780547000 Industrial Park Building no. B 2 Kalyan- Involce No Padgha road, Kalyan- Padgha road, Village- FAA9031900407482 DI:14-07-2018 www.flipkart.com/supportion goods soldare intended for end user consumption
se Ph:1800 208 9898 Shipping Address Billing Address Divya Jyoti Divya Jyoti -602, Prakrill Society, 1-602, Prakrili Society, BanerOppsite to New Era BanerOppsite to New Era BakeryMaharashtra BakeryMaharashtra Pune-411045 Pune-411045 Maharashtra Maharashtra 15888 Product CGST Total Price Oty DSLR & MWD60665 /Nikon 4920.78 4920.7844990 35148.44 T HSN: 85258020 14.006 14 046 8623616,22844642, Total 4920.78 4920.7844990 1 35148.44 page 1 of 1 14


In [22]:
txt2= '''
Setting `pad_token_id` to `eos_token_id`:151646 for open-end generation.
<|input|>
### Template:
{
    "Customer_Info": {
        "Name": "",
        "Shipping Address": "",
        "Phone": ""
    },
    "Product": {
        "Product_Name": "",
        "Price": ""
    }
}
### Text:
PL:F32771816 IT:1 Tax Invoice S:S444795365[1] Registered Office :: Tech Connect Retail Pvt. Ltd., 502 - 503, 5th Floor, Prospect Chanmers, Dr. d n Road, Fort Mumbai, Mumbai, MAHARASHTRA, INDIA-400001 Ordered Through Flipkart Sold by Tech Connect Retail Pvt. Ltd., GSTIN 27AAICA 4872D 120 Avaashya CCI Logistics Ltd Renaissance 0D112B39788780547000 Industrial Park Building no. B 2 Kalyan- Involce No Padgha road, Kalyan- Padgha road, Village- FAA9031900407482 DI:14-07-2018 www.flipkart.com/supportion goods soldare intended for end user consumption
se Ph:1800 208 9898 Shipping Address Billing Address Divya Jyoti Divya Jyoti -602, Prakrill Society, 1-602, Prakrili Society, BanerOppsite to New Era BanerOppsite to New Era BakeryMaharashtra BakeryMaharashtra Pune-411045 Pune-411045 Maharashtra Maharashtra 15888 Product CGST Total Price Oty DSLR & MWD60665 /Nikon 4920.78 4920.7844990 35148.44 T HSN: 85258020 14.006 14 046 8623616,22844642, Total 4920.78 4920.7844990 1 35148.44 page 1 of 1 14
</output/>
{
    "Customer_Info": {
        "Name": "Divya Jyoti",
        "Shipping Address": "1-602, Prakrili Society, BanerOppsite to New Era BakeryMaharashtra, Pune- 411045, Maharashtra",
        "Phone": ""
    },
    "Product": {
        "Product_Name": "DSLR & MWD60665 /Nikon",
        "Price": "4920.78"
    }
}
'''



PL:F32771816 IT:1 Tax Invoice S:S444795365[1] Registered Office :: Tech Connect Retail Pvt. Ltd., 502 - 503, 5th Floor, Prospect Chanmers, Dr. d n Road, Fort Mumbai, Mumbai, MAHARASHTRA, INDIA-400001 Ordered Through Flipkart Sold by Tech Connect Retail Pvt. Ltd., GSTIN 27AAICA 4872D 120 Avaashya CCI Logistics Ltd Renaissance 0D112B39788780547000 Industrial Park Building no. B 2 Kalyan- Involce No Padgha road, Kalyan- Padgha road, Village- FAA9031900407482 DI:14-07-2018 www.flipkart.com/supportion goods soldare intended for end user consumption
se Ph:1800 208 9898 Shipping Address Billing Address Divya Jyoti Divya Jyoti -602, Prakrill Society, 1-602, Prakrili Society, BanerOppsite to New Era BanerOppsite to New Era BakeryMaharashtra BakeryMaharashtra Pune-411045 Pune-411045 Maharashtra Maharashtra 15888 Product CGST Total Price Oty DSLR & MWD60665 /Nikon 4920.78 4920.7844990 35148.44 T HSN: 85258020 14.006 14 046 8623616,22844642, Total 4920.78 4920.7844990 1 35148.44 page 1 of 1 14
</

In [13]:
# Define schema for the bill receipt extraction
schema = """
{
  "Customer_Info": {
    "Name": "",
    "Shipping Address": "",
    "Phone": ""
  },
  "Product": {
    "Product_Name": "",
    "Price": ""
  }
}
"""

In [17]:
# Extract structured information
prediction = predict_NuExtract(model, tokenizer, text, schema)
print(prediction)


#To format extracted details
def extract_text_after_template(text):
    # Split the text to extract content after '### Text:'
    if "### Text:" in text:
        extracted_text = text.split("### Text:")[1]
        return extracted_text
    return extracted_text

output=extract_text_after_template(prediction)
print(output)


Setting `pad_token_id` to `eos_token_id`:151646 for open-end generation.
<|input|>
### Template:
{
    "Customer_Info": {
        "Name": "",
        "Shipping Address": "",
        "Phone": ""
    },
    "Product": {
        "Product_Name": "",
        "Price": ""
    }
}
### Text:
PL:F32771816 IT:1 Tax Invoice S:S444795365[1] Registered Office :: Tech Connect Retail Pvt. Ltd., 502 - 503, 5th Floor, Prospect Chanmers, Dr. d n Road, Fort Mumbai, Mumbai, MAHARASHTRA, INDIA-400001 Ordered Through Flipkart Sold by Tech Connect Retail Pvt. Ltd., GSTIN 27AAICA 4872D 120 Avaashya CCI Logistics Ltd Renaissance 0D112B39788780547000 Industrial Park Building no. B 2 Kalyan- Involce No Padgha road, Kalyan- Padgha road, Village- FAA9031900407482 DI:14-07-2018 www.flipkart.com/supportion goods soldare intended for end user consumption
se Ph:1800 208 9898 Shipping Address Billing Address Divya Jyoti Divya Jyoti -602, Prakrill Society, 1-602, Prakrili Society, BanerOppsite to New Era BanerOppsite to